In [ ]:
import numpy as np
import pandas as pd

from src.scripts.read_data import ReadData

In [ ]:
dataframes_path = "/data/processed/data_frames"
world_data_path = "/data/raw/daily_world_en_csv"

world_data_sentiments_raw = pd.read_parquet(f"{dataframes_path}/world_data_sentiments_raw.parquet")
read_data_world = ReadData(world_data_path, ['id', 'text', 'author_id'], filter_tweets=True)

In [ ]:
read_data_world.read_csvs_and_combine_data()
read_data_world.data = read_data_world.data.drop(columns=['text'])

/usr/lib/python3.7/multiprocessing/pool.py:44: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  return list(map(*args))
/usr/lib/python3.7/multiprocessing/pool.py:44: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  return list(map(*args))
/usr/lib/python3.7/multiprocessing/pool.py:44: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  return list(map(*args))
/usr/lib/python3.7/multiprocessing/pool.py:44: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  return list(map(*args))
100%|██████████| 73/73 [01:14<00:00,  1.02s/it]


In [ ]:
read_data_world.data['author_id'] = read_data_world.data['author_id'].astype('str')
world_data_sentiments_raw = world_data_sentiments_raw.join(read_data_world.data.set_index('id'), on='id').dropna()
world_data_sentiments_raw = world_data_sentiments_raw[world_data_sentiments_raw['id'].isin(read_data_world.data['id'].unique())]

In [ ]:
world_data_sentiments_raw['created_at'] = pd.to_datetime(world_data_sentiments_raw['created_at'].apply(lambda x: x[:10]))

authorid_date_sentiment = world_data_sentiments_raw[['author_id', 'created_at', 'Anti', 'Pro']]

In [ ]:
authorid_date_sentiment.reset_index(drop=True).to_parquet(f"{dataframes_path}/authorid_date_anti_pro.parquet", index=False)

In [ ]:
authorid_date_sentiment = authorid_date_sentiment.iloc[np.max(authorid_date_sentiment.iloc[:, 2:].values, axis=1) >= 0.99, :]
authorid_date_sentiment['sentiment'] = np.argmax(authorid_date_sentiment[['Anti', 'Pro']].values, axis=1)
authorid_date_sentiment = authorid_date_sentiment.drop(columns=['Anti', 'Pro'])

authorid_date_sentiment_TweetCounts = authorid_date_sentiment.value_counts().reset_index(name='tweet_counts')
authorid_date_sentiment_TweetCounts.to_parquet(f"{dataframes_path}/authorid_date_sentiment_tweetcounts.parquet", index=False)